# Tests for subareas at levels lower than 8 (default) and areas not entirely included in image acquisition (Supamammillary nucleus)

This notebook performs an analysis of number of detected cells at levels with a finer granularity of our default (level 8 of the Allen Brain Atlas). It tests the number of detected cells across groups for all areas, and returns areas with a significant difference, for all experimental groups (c57 with and without father).

In a second section, we consider only the Suprammamillary nucleus. We reanalyse data obtained at level 8 and do not consider subjects where the SUM area has been cut out.

In [1]:
# !pip install odfpy
# !pip install seaborn
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import yaml
from yaml import Loader
import numpy as np
import analyze_cells_energy as ace
import utils
from scipy.stats import ttest_ind, mannwhitneyu
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
# load query file where we added volumes for each area
volumes = ace.clean_volumes_database()

In [3]:
def find_significant_areas(dictionary, experimental_groups, value, test='mannwhitneyu', alpha=0.05):
    """
    Find significant areas between experimental groups for a given value using statistical tests.

    This function calculates the values across experimental groups, performs statistical tests,
    and identifies significant areas where the p-value is below a specified threshold (alpha).

    Parameters:
    dictionary (dict): Nested dictionary with results data.
    experimental_groups (dict): Dictionary specifying the experimental groups.
    value (str): The value to be tested across groups.
    test (str): The statistical test to be used (default is 'mannwhitneyu').
    alpha (float): The significance level threshold (default is 0.05).

    Returns:
    pd.DataFrame: DataFrame containing significant areas for each comparison between groups.
    """
    # Calculate the value across experimental groups
    dfs = ace.calculate_value_across_groups(
        experimental_groups=experimental_groups,
        dict_results_across_mice=dictionary,
        value=value
    )

    # Perform statistical tests across groups
    df_ttest = ace.test_across_groups(
        dfs,
        test=test,
        groups=list(experimental_groups.keys())
    )

    # Get the columns representing p-values from the statistical tests
    columns = df_ttest.loc[:, df_ttest.columns != 'area'].columns

    # Initialize an empty DataFrame to store significant areas
    df_sigareas = pd.DataFrame()

    # Iterate over each column (comparison)
    for col in columns:
        # Sort the DataFrame by the current column (p-value) and select the 'area' and p-value column
        df = df_ttest.sort_values(by=col)[['area', col]]

        # Concatenate significant areas (p-value < alpha) to the result DataFrame
        df_sigareas = pd.concat([df_sigareas, df[df[col] < alpha]['area'].reset_index(drop=True)], axis=1)

    # Rename columns to remove the 'pval_' prefix
    df_sigareas.columns = [col.replace('pval_', '') for col in columns]

    return df_sigareas

# Part 1: Data analysis

## c57 WITH FATHER

### Data loading

In [4]:
root_directory = '/home/stella/Documents/Torino/projects/'
experiment = 'SexualImprinting'
experimental_group = 'C57_MaleUrine_Exposure_cFos'
data_directory = root_directory + experiment + '/' \
                + experimental_group + '/'
subjects = [name for name in os.listdir(data_directory) \
            if os.path.isdir(os.path.join(data_directory, name))]
threshold = (300, 2500)
experimental_groups_wf = utils.divide_in_exp_groups(list_subjects=subjects)

In [5]:
def load_subject(subject, data_directory, threshold1hemisphere, threshold2hemispheres, 
                 thresholdF6Unfam, thresholdBL48Unfam):
    if subject[0] == 'F':
        if subject =='F6Unfam':
            file_suffix = 'cells_' + str(thresholdF6Unfam)
        else:
            file_suffix = 'cells_' + str(threshold1hemisphere)
    else:
        if subject == 'BL48Unfam':
            file_suffix = 'cells_' + str(thresholdBL48Unfam)
        else:
            file_suffix = 'cells_' + str(threshold2hemispheres)
            
    df_mouse = pd.read_csv(data_directory + subject + '/' + file_suffix + '.csv')
    if subject[0]=='B':
        df_mouse = ace.reformat_df_mouse(df=df_mouse[df_mouse[' x']>228].reset_index(drop=True))
    else:
        df_mouse = ace.reformat_df_mouse(df=df_mouse)
    
    return df_mouse

In [6]:
# for level in [9,10,11]:
#     print(level)
#     dict_results_across_mice = {subject: ace.calculate_cells_energy_per_level(
#         df_mouse=load_subject(subject=subject, 
#                               data_directory=data_directory, 
#                               threshold1hemisphere=300, 
#                               threshold2hemispheres=2500,
#                               thresholdF6Unfam=600,
#                               thresholdBL48Unfam=3000), 
#         vol=volumes, 
#         level=level) for subject in subjects}
#     np.save('dict_results/newvolumes/dict_results_across_mice_c57_merged_hemispheres_level'+str(level)+'.npy', 
#             dict_results_across_mice)

### Level 9: ttest and kruskal wallis test

In [7]:
level9 = np.load('dict_results/newvolumes/dict_results_across_mice_c57_merged_hemispheres_level9.npy',
                 allow_pickle=True).item()
## ttest
find_significant_areas(dictionary=level9, 
                       experimental_groups=experimental_groups_wf,
                       value='n_cells',
                       test='mannwhitneyu',
                      alpha=0.05)

,Control_vs_Fam,Control_vs_Unfam,Fam_vs_Unfam
0,Cortical amygdalar area posterior part,Cortical amygdalar area posterior part,NaN
1,Posterior pretectal nucleus,Basolateral amygdalar nucleus ventral part,NaN
2,Tuberomammillary nucleus ventral part,Orbital area medial part,NaN
3,Endopiriform nucleus ventral part,Endopiriform nucleus dorsal part,NaN
4,Entorhinal area medial part ventral zone,Periventricular hypothalamic nucleus anterior ...,NaN
5,Basolateral amygdalar nucleus anterior part,Tuberomammillary nucleus ventral part,NaN
6,Endopiriform nucleus dorsal part,Basolateral amygdalar nucleus anterior part,NaN
7,NaN,Endopiriform nucleus ventral part,NaN
8,NaN,Periventricular hypothalamic nucleus intermedi...,NaN


In [8]:
# save results to csv file
find_significant_areas(dictionary=level9, 
                       experimental_groups=experimental_groups_wf,
                       value='n_cells',
                       test='mannwhitneyu',
                      alpha=0.05).to_csv('non_param_test_c57WF_level9.csv')

In [9]:
#ks-test
ace.kruskal_per_area(dictionary=level9,
                     experimental_groups=utils.divide_in_exp_groups(list_subjects=list(level9.keys())),
                     value='n_cells',
                     level=9)

{'Orbital area medial part': (KruskalResult(statistic=7.288644688644695, pvalue=0.026139117174669622),
            Control       Fam     Unfam
  Control  1.000000  0.158366  0.020893
  Fam      0.158366  1.000000  0.263033
  Unfam    0.020893  0.263033  1.000000),
 'Cortical amygdalar area posterior part': (KruskalResult(statistic=14.90769230769233, pvalue=0.00057920959339604),
            Control       Fam     Unfam
  Control  1.000000  0.001175  0.001504
  Fam      0.001175  1.000000  0.718521
  Unfam    0.001504  0.718521  1.000000),
 'Basolateral amygdalar nucleus ventral part': (KruskalResult(statistic=8.157926186673627, pvalue=0.01692500620888636),
            Control       Fam     Unfam
  Control  1.000000  0.062722  0.014867
  Fam      0.062722  1.000000  0.434599
  Unfam    0.014867  0.434599  1.000000)}

### Level 10: ttest and kruskal wallis test

No areas are found significant at level 10, since ClearMap does not detect cells in areas at that level.

In [10]:
level10 = np.load('dict_results/newvolumes/dict_results_across_mice_c57_merged_hemispheres_level10.npy',
                 allow_pickle=True).item()
find_significant_areas(dictionary=level10, 
                       experimental_groups=experimental_groups_wf,
                       value='n_cells',
                       test='mannwhitneyu',
                      alpha=0.05)

,Control_vs_Fam,Control_vs_Unfam,Fam_vs_Unfam


In [11]:
# save results to csv file
find_significant_areas(dictionary=level10, 
                       experimental_groups=experimental_groups_wf,
                       value='n_cells',
                       test='mannwhitneyu',
                      alpha=0.05).to_csv('non_param_test_c57WF_level10.csv')

In [12]:
#ks-test
ace.kruskal_per_area(dictionary=level10,
                     experimental_groups=utils.divide_in_exp_groups(list_subjects=list(level10.keys())),
                     value='n_cells',
                     level=10)

{}

### Level 11

In [13]:
level11 = np.load('dict_results/newvolumes/dict_results_across_mice_c57_merged_hemispheres_level11.npy',
                 allow_pickle=True).item()
find_significant_areas(dictionary=level11, 
                       experimental_groups=experimental_groups_wf,
                       value='n_cells',
                       test='mannwhitneyu',
                      alpha=0.05)

,Control_vs_Fam,Control_vs_Unfam,Fam_vs_Unfam
0,Accessory olfactory bulb granular layer,Accessory olfactory bulb granular layer,Posterior auditory area layer 6b
1,Accessory olfactory bulb mitral layer,Accessory olfactory bulb mitral layer,Primary somatosensory area mouth layer 6b
2,Infralimbic area layer 5,Infralimbic area layer 5,Dorsal auditory area layer 5
3,Posterolateral visual area layer 4,Orbital area medial part layer 5,NaN
4,Posterolateral visual area layer 5,Nucleus of the lateral olfactory tract layer 3,NaN
5,Nucleus of the lateral olfactory tract layer 3,Orbital area medial part layer 2/3,NaN
6,Visceral area layer 1,Posterior auditory area layer 6b,NaN
7,Perirhinal area layer 1,Posterolateral visual area layer 5,NaN
8,Orbital area medial part layer 2/3,Lateral visual area layer 5,NaN
9,NaN,Dentate gyrus molecular layer,NaN


In [14]:
# save results to csv file
find_significant_areas(dictionary=level11, 
                       experimental_groups=experimental_groups_wf,
                       value='n_cells',
                       test='mannwhitneyu',
                      alpha=0.05).to_csv('non_param_test_c57WF_level11.csv')

In [15]:
#ks-test
ace.kruskal_per_area(dictionary=level11,
                     experimental_groups=utils.divide_in_exp_groups(list_subjects=list(level11.keys())),
                     value='n_cells',
                     level=11)

{'Posterior auditory area layer 6b': (KruskalResult(statistic=6.826310380267206, pvalue=0.03293711339990913),
            Control       Fam     Unfam
  Control  1.000000  0.761026  0.077172
  Fam      0.761026  1.000000  0.046227
  Unfam    0.077172  0.046227  1.000000),
 'Infralimbic area layer 5': (KruskalResult(statistic=7.550383525503225, pvalue=0.022932692584018765),
            Control       Fam     Unfam
  Control  1.000000  0.023015  0.023015
  Fam      0.023015  1.000000  0.928188
  Unfam    0.023015  0.928188  1.000000),
 'Orbital area medial part layer 5': (KruskalResult(statistic=6.820653649056531, pvalue=0.033030403465604416),
            Control       Fam     Unfam
  Control  1.000000  0.211284  0.028398
  Fam      0.211284  1.000000  0.211284
  Unfam    0.028398  0.211284  1.000000),
 'Accessory olfactory bulb granular layer': (KruskalResult(statistic=10.116061928647447, pvalue=0.006358066455242902),
            Control       Fam     Unfam
  Control  1.000000  0.008682  

## c57 WITHOUT FATHER

### Data loading

In [17]:
root_directory = '/home/stella/Documents/Torino/projects/'
experiment = 'SexualImprinting'
experimental_group = 'BL_woF'
data_directory = root_directory + experiment + '/' \
                + experimental_group + '/'
merged_controls = False
    
subjects = [name for name in os.listdir(data_directory) \
                if os.path.isdir(os.path.join(data_directory, name))]
experimental_groups_wof = utils.divide_in_exp_groups(list_subjects=subjects)

In [18]:
def load_subject(subject, data_directory, threshold1, threshold2):
    file_suffix = 'cells' + str(threshold2) if subject[0] == 'BL36Fam' else 'cells' + str(threshold1)
    df_mouse = pd.read_csv(data_directory + subject + '/' + file_suffix + '.csv')
    df_mouse = ace.reformat_df_mouse(df=df_mouse)
    
    return df_mouse

In [19]:
# for level in [9,10,11]:
#     print(level)
#     dict_results_across_mice = {subject: ace.calculate_cells_energy_per_level(
#         df_mouse=load_subject(subject=subject, 
#                               data_directory=data_directory, 
#                               threshold1=2000, 
#                               threshold2=1500), 
#         vol=volumes, 
#         level=level) for subject in subjects}
#     np.save('dict_results/newvolumes/dict_results_across_mice_BL_woF_level'+str(level)+'.npy', dict_results_across_mice)

### Level 9

In [20]:
level9 = np.load('dict_results/newvolumes/dict_results_across_mice_BL_woF_level9.npy',
                 allow_pickle=True).item()
find_significant_areas(dictionary=level9, 
                       experimental_groups=experimental_groups_wof,
                       value='n_cells',
                       test='mannwhitneyu',
                      alpha=0.05)

,Control_vs_Fam,Control_vs_Unfam,Fam_vs_Unfam
0,Cortical amygdalar area posterior part,Cortical amygdalar area posterior part,Basolateral amygdalar nucleus ventral part
1,Posterior pretectal nucleus,Lateral septal nucleus ventral part,Ventral posteromedial nucleus of the thalamus ...
2,Ventral posteromedial nucleus of the thalamus,Entorhinal area medial part ventral zone,Medial septal nucleus
3,Primary somatosensory area upper limb,Medial septal nucleus,Agranular insular area ventral part
4,NaN,Tuberomammillary nucleus ventral part,Lateral septal nucleus ventral part
5,NaN,Basolateral amygdalar nucleus ventral part,Cortical amygdalar area posterior part
6,NaN,Lateral septal nucleus rostral (rostroventral)...,Primary somatosensory area mouth
7,NaN,NaN,Tuberomammillary nucleus ventral part
8,NaN,NaN,Basolateral amygdalar nucleus posterior part


In [21]:
# save results to csv file
find_significant_areas(dictionary=level9, 
                       experimental_groups=experimental_groups_wof,
                       value='n_cells',
                       test='mannwhitneyu',
                      alpha=0.05).to_csv('non_param_test_c57WOF_level9.csv')

In [22]:
#ks-test
ace.kruskal_per_area(dictionary=level9,
                     experimental_groups=utils.divide_in_exp_groups(list_subjects=list(level9.keys())),
                     value='n_cells',
                     level=9)

{'Cortical amygdalar area posterior part': (KruskalResult(statistic=14.98522167487684, pvalue=0.0005571863368546974),
            Control       Fam     Unfam
  Control  1.000000  0.038654  0.000327
  Fam      0.038654  1.000000  0.081909
  Unfam    0.000327  0.081909  1.000000),
 'Entorhinal area medial part ventral zone': (KruskalResult(statistic=9.427074774034505, pvalue=0.008972980471623404),
            Control       Fam     Unfam
  Control  1.000000  0.137763  0.006416
  Fam      0.137763  1.000000  0.141972
  Unfam    0.006416  0.141972  1.000000),
 'Basolateral amygdalar nucleus ventral part': (KruskalResult(statistic=6.572689882094864, pvalue=0.03739026356100799),
            Control       Fam     Unfam
  Control  1.000000  0.748466  0.079155
  Fam      0.748466  1.000000  0.049943
  Unfam    0.079155  0.049943  1.000000),
 'Lateral septal nucleus ventral part': (KruskalResult(statistic=10.985134393856283, pvalue=0.004117260776017655),
            Control       Fam     Unfam
  

### Level 10

In [23]:
level10 = np.load('dict_results/newvolumes/dict_results_across_mice_BL_woF_level10.npy',
                 allow_pickle=True).item()
find_significant_areas(dictionary=level10, 
                       experimental_groups=experimental_groups_wof,
                       value='n_cells',
                       test='mannwhitneyu',
                      alpha=0.05)

,Control_vs_Fam,Control_vs_Unfam,Fam_vs_Unfam


In [24]:
# save results to csv file
find_significant_areas(dictionary=level10, 
                       experimental_groups=experimental_groups_wof,
                       value='n_cells',
                       test='mannwhitneyu',
                      alpha=0.05).to_csv('non_param_test_c57WOF_level10.csv')

In [25]:
#ks-test
ace.kruskal_per_area(dictionary=level10,
                     experimental_groups=utils.divide_in_exp_groups(list_subjects=list(level10.keys())),
                     value='n_cells',
                     level=10)

{}

In [26]:
# df_control_cell_count, df_fam_cell_count, df_unfam_cell_count = \
# ace.calculate_value_across_groups(experimental_groups=experimental_groups_wof, 
#                               dict_results_across_mice=level10, 
#                               value='n_cells')

### Level 11

In [27]:
level11 = np.load('dict_results/newvolumes/dict_results_across_mice_BL_woF_level11.npy',
                 allow_pickle=True).item()
find_significant_areas(dictionary=level11, 
                       experimental_groups=experimental_groups_wof,
                       value='n_cells',
                       test='mannwhitneyu',
                      alpha=0.05)

,Control_vs_Fam,Control_vs_Unfam,Fam_vs_Unfam
0,Accessory olfactory bulb mitral layer,Accessory olfactory bulb mitral layer,Primary somatosensory area mouth layer 1
1,Accessory olfactory bulb granular layer,Accessory olfactory bulb granular layer,Primary somatosensory area trunk layer 1
2,Agranular insular area ventral part layer 6b,Primary auditory area layer 1,Anteromedial visual area layer 5
3,Visceral area layer 5,Anteromedial visual area layer 5,Agranular insular area posterior part layer 6b
4,Perirhinal area layer 2/3,Primary visual area layer 2/3,Primary somatosensory area upper limb layer 1
5,Infralimbic area layer 6b,Temporal association areas layer 2/3,Agranular insular area dorsal part layer 2/3
6,Posterior auditory area layer 1,Retrosplenial area lateral agranular part layer 1,Primary motor area Layer 1
7,Dorsal auditory area layer 5,Anteromedial visual area layer 2/3,Retrosplenial area dorsal part layer 6b
8,Visceral area layer 2/3,NaN,Agranular insular area ventral part layer 5
9,Primary somatosensory area trunk layer 1,NaN,Gustatory areas layer 4


In [28]:
# save results to csv file
find_significant_areas(dictionary=level11, 
                       experimental_groups=experimental_groups_wof,
                       value='n_cells',
                       test='mannwhitneyu',
                      alpha=0.05).to_csv('non_param_test_c57WOF_level11.csv')

In [29]:
#ks-test
ace.kruskal_per_area(dictionary=level11,
                     experimental_groups=utils.divide_in_exp_groups(list_subjects=list(level11.keys())),
                     value='n_cells',
                     level=11)

{'Primary somatosensory area upper limb layer 1': (KruskalResult(statistic=6.011579670329674, pvalue=0.04949964232630368),
            Control       Fam     Unfam
  Control  1.000000  0.070951  0.923289
  Fam      0.070951  1.000000  0.070951
  Unfam    0.923289  0.070951  1.000000),
 'Anteromedial visual area layer 5': (KruskalResult(statistic=6.391542568137109, pvalue=0.04093494072964938),
            Control       Fam     Unfam
  Control  1.000000  0.541695  0.133438
  Fam      0.541695  1.000000  0.042695
  Unfam    0.133438  0.042695  1.000000),
 'Agranular insular area dorsal part layer 2/3': (KruskalResult(statistic=5.9934228555768625, pvalue=0.04995106624903229),
            Control       Fam     Unfam
  Control  1.000000  0.056658  0.938673
  Fam      0.056658  1.000000  0.056658
  Unfam    0.938673  0.056658  1.000000),
 'Accessory olfactory bulb granular layer': (KruskalResult(statistic=12.551671232876707, pvalue=0.001881218419775376),
            Control       Fam     Unfam

# Adapted tests for SUM


## c57 WITH FATHER

In [30]:
c57_wf = np.load('dict_results/newvolumes/dict_results_across_mice_c57_merged_hemispheres_F6BL48.npy', 
                                   allow_pickle=True).item()
# remove animals that do not include SUM 
c57_wf.pop('F1Control')
c57_wf.pop('F9Fam')
experimental_groups = utils.divide_in_exp_groups(list_subjects=c57_wf.keys())

### NON PARAMETRIC TEST 

In [31]:
df_control_cell_count, df_fam_cell_count, df_unfam_cell_count = \
ace.calculate_value_across_groups(experimental_groups=experimental_groups, 
                              dict_results_across_mice=c57_wf, 
                              value='n_cells')

df_ttest_ncells = ace.test_across_groups([df_control_cell_count,df_fam_cell_count,df_unfam_cell_count], 
                                         test='mannwhitneyu')

df_ttest_not_null_ncells = df_ttest_ncells.dropna(axis=0,
                                   how='all',
                                   subset=['pval_Control_vs_Fam',
                                          'pval_Control_vs_Unfam',
                                          'pval_Fam_vs_Unfam'])

df_ttest_not_null_ncells.set_index('area').loc['Supramammillary nucleus']

pval_Control_vs_Fam       0.238595
pval_Control_vs_Unfam     0.127201
pval_Fam_vs_Unfam        0.0985791
Name: Supramammillary nucleus, dtype: object

### KRUSKAL WALLIS TEST

In [32]:
ks_wf = ace.kruskal_per_area(dictionary=c57_wf,
                     experimental_groups=utils.divide_in_exp_groups(list_subjects=list(c57_wf.keys())),
                     value='n_cells',
                     level=8)

In [33]:
ks_wf.keys()

dict_keys(['Main olfactory bulb', 'Accessory olfactory bulb', 'Cortical amygdalar area', 'Piriform-amygdalar area', 'Basomedial amygdalar nucleus', 'Posterior amygdalar nucleus', 'Medial amygdalar nucleus', 'Bed nuclei of the stria terminalis', 'Arcuate hypothalamic nucleus', 'Posterodorsal preoptic nucleus', 'Periventricular hypothalamic nucleus posterior part', 'Medial preoptic nucleus', 'Ventral premammillary nucleus', 'Tuberal nucleus', 'Lateral terminal nucleus of the accessory optic tract'])

## c57 WITHOUT FATHER

In [34]:
c57_wof = np.load('dict_results/newvolumes/dict_results_across_mice_BL_woF2000_36.npy', 
                                   allow_pickle=True).item()
c57_wof.pop('BL21Fam')
c57_wof.pop('BL24Fam')
c57_wof.pop('BL36Fam')
c57_wof.pop('BL37Fam')
c57_wof.pop('BL31Unfam')
c57_wof.pop('BL35Unfam')
experimental_groups = utils.divide_in_exp_groups(list_subjects=c57_wof.keys())

### NON PARAMETRIC TEST

In [35]:
df_control_cell_count, df_fam_cell_count, df_unfam_cell_count = \
ace.calculate_value_across_groups(experimental_groups=experimental_groups, 
                              dict_results_across_mice=c57_wof, 
                              value='n_cells')

df_ttest_ncells = ace.test_across_groups([df_control_cell_count,df_fam_cell_count,df_unfam_cell_count], 
                                         test='mannwhitneyu')

df_ttest_not_null_ncells = df_ttest_ncells.dropna(axis=0,
                                   how='all',
                                   subset=['pval_Control_vs_Fam',
                                          'pval_Control_vs_Unfam',
                                          'pval_Fam_vs_Unfam'])

df_ttest_not_null_ncells.set_index('area').loc['Supramammillary nucleus']

pval_Control_vs_Fam      0.0775629
pval_Control_vs_Unfam    0.0517809
pval_Fam_vs_Unfam         0.280638
Name: Supramammillary nucleus, dtype: object

### KRUSKAL WALLIS TEST

In [36]:
ks_wof = ace.kruskal_per_area(dictionary=c57_wof,
                     experimental_groups=utils.divide_in_exp_groups(list_subjects=list(c57_wof.keys())),
                     value='n_cells',
                     level=8)

In [37]:
ks_wof

{'Main olfactory bulb': (KruskalResult(statistic=11.826086956521735, pvalue=0.002703944953989953),
            Control      Fam     Unfam
  Control  1.000000  0.05478  0.002112
  Fam      0.054780  1.00000  0.295770
  Unfam    0.002112  0.29577  1.000000),
 'Accessory olfactory bulb': (KruskalResult(statistic=14.95607344632769, pvalue=0.0005653662972509756),
            Control       Fam     Unfam
  Control  1.000000  0.007553  0.000761
  Fam      0.007553  1.000000  0.560335
  Unfam    0.000761  0.560335  1.000000),
 'Anterior olfactory nucleus': (KruskalResult(statistic=7.947628458498016, pvalue=0.018801582648680502),
            Control       Fam     Unfam
  Control  1.000000  0.201577  0.014841
  Fam      0.201577  1.000000  0.269180
  Unfam    0.014841  0.269180  1.000000),
 'Cortical amygdalar area': (KruskalResult(statistic=14.778656126482218, pvalue=0.0006178109439535902),
            Control       Fam     Unfam
  Control  1.000000  0.024592  0.000484
  Fam      0.024592  1.000